# Mastering Polars queries with contexts and more complex expressions

In [1]:
import polars as pl
import numpy as np

In [2]:
df = pl.scan_csv('hotels-europe_price.csv')

## Some basic Dataframe inspection

In [3]:
df.columns

['hotel_id',
 'price',
 'offer',
 'offer_cat',
 'year',
 'month',
 'weekend',
 'holiday',
 'nnights',
 'scarce_room']

In [4]:
df.head().collect()

hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room
i64,i64,i64,str,i64,i64,i64,i64,i64,i64
1,172,0,"""0% no offer""",2017,11,1,0,1,0
1,122,1,"""15-50% offer""",2018,1,1,0,1,0
1,122,1,"""15-50% offer""",2017,12,0,1,1,0
1,552,1,"""1-15% offer""",2017,12,0,1,4,0
1,122,1,"""15-50% offer""",2018,2,1,0,1,0


In [5]:
df.collect().describe()

describe,hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room
str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""count""",148021.0,148021.0,148021.0,"""148021""",148021.0,148021.0,148021.0,148021.0,148021.0,148021.0
"""null_count""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",10752.712777,179.009674,0.583667,null,2017.579553,6.862776,0.665156,0.218489,1.330804,0.527959
"""std""",6545.319965,256.056927,0.492952,null,0.493632,4.184376,0.471938,0.413223,0.939674,0.499219
"""min""",1.0,5.0,0.0,"""0% no offer""",2017.0,1.0,0.0,0.0,1.0,0.0
"""25%""",5227.0,74.0,0.0,null,2017.0,3.0,0.0,0.0,1.0,0.0
"""50%""",10403.0,113.0,1.0,null,2018.0,6.0,1.0,0.0,1.0,1.0
"""75%""",15132.0,192.0,1.0,null,2018.0,11.0,1.0,0.0,1.0,1.0
"""max""",22902.0,39996.0,1.0,"""75%+ offer """,2018.0,12.0,1.0,1.0,4.0,1.0


## Part 1: Defining the context

In [6]:
df.select([
    pl.col('hotel_id')
]).collect()

hotel_id
i64
1
1
1
1
1
1
2
2
2


In [7]:
df.select([
    pl.col('hotel_id'),
    pl.col('year'),
    pl.col('price'),
    pl.col('offer_cat')
]).collect()

hotel_id,year,price,offer_cat
i64,i64,i64,str
1,2017,172,"""0% no offer"""
1,2018,122,"""15-50% offer"""
1,2017,122,"""15-50% offer"""
1,2017,552,"""1-15% offer"""
1,2018,122,"""15-50% offer"""
1,2017,114,"""15-50% offer"""
2,2017,119,"""0% no offer"""
2,2017,119,"""0% no offer"""
2,2017,547,"""0% no offer"""


### Filter

In [8]:
df.filter(
    (pl.col('year') == 2017)
).collect()

hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room
i64,i64,i64,str,i64,i64,i64,i64,i64,i64
1,172,0,"""0% no offer""",2017,11,1,0,1,0
1,122,1,"""15-50% offer""",2017,12,0,1,1,0
1,552,1,"""1-15% offer""",2017,12,0,1,4,0
1,114,1,"""15-50% offer""",2017,11,0,0,1,0
2,119,0,"""0% no offer""",2017,11,0,0,1,0
2,119,0,"""0% no offer""",2017,12,0,1,1,0
2,547,0,"""0% no offer""",2017,12,0,1,4,0
3,118,1,"""15-50% offer""",2017,12,0,1,1,0
3,217,0,"""0% no offer""",2017,11,1,0,1,1


In [9]:
df.filter(
    (pl.col('year') == 2017) &
    (pl.col('price') > 2500)
).collect()

hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room
i64,i64,i64,str,i64,i64,i64,i64,i64,i64
5,3087,1,"""15-50% offer""",2017,12,0,1,4,1
113,2940,1,"""1-15% offer""",2017,12,0,1,4,0
118,2998,0,"""0% no offer""",2017,12,0,1,4,1
197,3308,0,"""0% no offer""",2017,12,0,1,4,0
299,3620,1,"""1-15% offer""",2017,12,0,1,4,1
319,2704,1,"""15-50% offer""",2017,12,0,1,4,1
869,2886,0,"""0% no offer""",2017,12,0,1,4,1
922,2522,0,"""0% no offer""",2017,12,0,1,4,1
1084,3499,1,"""15-50% offer""",2017,12,0,1,4,1


In [10]:
df.filter(
    (pl.col('year') != 2018) &
    (pl.col('price') > 2500) & 
    (pl.col('price') < 2600) &
    (
        (pl.col('offer_cat') == "50%-75% offer") |
        (pl.col('offer_cat') == "0% no offer")
    )
).collect()

hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room
i64,i64,i64,str,i64,i64,i64,i64,i64,i64
922,2522,0,"""0% no offer""",2017,12,0,1,4,1
1087,2580,0,"""0% no offer""",2017,12,0,1,4,0
3417,2507,0,"""0% no offer""",2017,12,0,1,4,1
7905,2512,0,"""0% no offer""",2017,12,0,1,4,1
8035,2507,0,"""0% no offer""",2017,12,0,1,4,0
8512,2548,1,"""50%-75% offer""",2017,12,0,1,4,0
9535,2509,0,"""0% no offer""",2017,12,0,1,4,1
18785,2509,0,"""0% no offer""",2017,12,0,1,4,1
22017,2531,0,"""0% no offer""",2017,12,0,1,4,1


### with_columns

In [11]:
df.with_columns(
    breakfast = pl.lit(1)
).collect()

hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room,breakfast
i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i32
1,172,0,"""0% no offer""",2017,11,1,0,1,0,1
1,122,1,"""15-50% offer""",2018,1,1,0,1,0,1
1,122,1,"""15-50% offer""",2017,12,0,1,1,0,1
1,552,1,"""1-15% offer""",2017,12,0,1,4,0,1
1,122,1,"""15-50% offer""",2018,2,1,0,1,0,1
1,114,1,"""15-50% offer""",2017,11,0,0,1,0,1
2,119,0,"""0% no offer""",2017,11,0,0,1,0,1
2,119,0,"""0% no offer""",2017,12,0,1,1,0,1
2,547,0,"""0% no offer""",2017,12,0,1,4,0,1


In [12]:
df.with_columns(
    random_breakfast = pl.Series(
        np.random.choice([0, 1], len(df.select(pl.col("hotel_id")).collect()))
    )
).collect()

# to create values for each row the length of the dataframe has to be determined. 
# selecting and collecting only 'hotel_id' takes 13ms, 
# selecting the whole dataframe (with pl.all() or pl.col('*')) takes 31ms

hotel_id,price,offer,offer_cat,year,month,weekend,holiday,nnights,scarce_room,random_breakfast
i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64
1,172,0,"""0% no offer""",2017,11,1,0,1,0,0
1,122,1,"""15-50% offer""",2018,1,1,0,1,0,1
1,122,1,"""15-50% offer""",2017,12,0,1,1,0,1
1,552,1,"""1-15% offer""",2017,12,0,1,4,0,0
1,122,1,"""15-50% offer""",2018,2,1,0,1,0,0
1,114,1,"""15-50% offer""",2017,11,0,0,1,0,1
2,119,0,"""0% no offer""",2017,11,0,0,1,0,0
2,119,0,"""0% no offer""",2017,12,0,1,1,0,0
2,547,0,"""0% no offer""",2017,12,0,1,4,0,0


### group_by

In [13]:
df.group_by('hotel_id').agg(
    pl.col('price').mean().alias('avg_price'),
).collect()

# you can add maintain_order=True to the groupby()
# this will ensure that the order remains similar
# to that in the dataframe. however the query becomes
# slower if you include it (see Reference guide)

hotel_id,avg_price
i64,f64
16832,315.0
7056,429.0
22696,123.7
19688,273.333333
6128,61.714286
17688,110.666667
22000,348.285714
7464,132.625
14072,274.0


In [14]:
df.group_by(['hotel_id', 'year'],maintain_order=True).agg(
    pl.col('price').mean().alias('avg_price'),
    pl.col('price').min().alias('min_price'),
    pl.col('price').max().alias('max_price'),
    pl.col('price').n_unique().alias('n_prices')
).head(15).collect()

hotel_id,year,avg_price,min_price,max_price,n_prices
i64,i64,f64,i64,i64,u32
1,2017,240.0,114,552,4
1,2018,122.0,122,122,1
2,2017,261.666667,119,547,2
3,2017,297.25,117,737,4
3,2018,114.0,114,114,1
4,2017,93.0,71,115,2
5,2018,809.0,696,922,2
5,2017,1347.25,690,3087,3
6,2018,252.0,216,324,2


## Part 2: Writing more complex queries

In [15]:
df_len = len(df.select(pl.col('hotel_id')).collect())

In [16]:
df.with_columns(
        is_breakfast = pl.Series(np.random.choice([0, 1], df_len)),
        is_premium_location = pl.Series(np.random.choice([0, 1], df_len)),
        total_price = pl.col('price') * pl.col('nnights')
    ).filter(
        (pl.col('is_breakfast') == 1) &
        (pl.col('is_premium_location') == 1) &
        (
            (pl.col('total_price') >= 1000) |
            (pl.col('price') > 1000)
        ) &
        (pl.col('year') == 2018) &
        (pl.col('month').is_in([1, 2, 3, 4, 5, 6]))
    ).select(
        'hotel_id',
        'year',
        'month',
        'price',
        'total_price',
        'is_breakfast',
        'is_premium_location',
        'offer_cat'
    ).sort(by='total_price', descending=True).collect()

hotel_id,year,month,price,total_price,is_breakfast,is_premium_location,offer_cat
i64,i64,i64,i64,i64,i64,i64,str
14739,2018,3,7674,7674,1,1,"""0% no offer"""
10945,2018,4,5000,5000,1,1,"""0% no offer"""
1087,2018,3,1723,1723,1,1,"""0% no offer"""
11432,2018,3,1551,1551,1,1,"""0% no offer"""
6400,2018,5,1499,1499,1,1,"""0% no offer"""
12644,2018,6,1485,1485,1,1,"""0% no offer"""
13360,2018,6,1463,1463,1,1,"""0% no offer"""
10374,2018,2,1462,1462,1,1,"""0% no offer"""
21182,2018,6,1427,1427,1,1,"""0% no offer"""
